<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_1_SRAL_advanced_data_access_eumdac.ipynb" target="_blank"><< Advanced SRAL data access with the EUMDAC client</a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn SRAL: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
Users should review the following notebooks for more information on setting up a credentials files for using the eumdac python library to retrieve SRAL data from the Data Store:
    
- **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store.ipynb</a>**

</div>
<hr>

# 2.2 Acquiring a full SRAL cycle

### Data used


| Product Description  | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:--------:|
| Sentinel-3 SRAL level-2 | EO:EUM:DAT:0415 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SR_2_WAT___NTC?query=SRAL&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SR_2_WAT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_2_WAT___" target="_blank">link</a>|

### Learning outcomes

At the end of this notebook you will know;
* <font color="#138D75">**Search**</font> for a time series of L2 SRAL data for a given cycle number using the EUMETSAT Data Store API client (`eumdac`)
* <font color="#138D75">**Download**</font> this time series

### Outline

The EUMETSAT Data Store offers many ways to interact with data in order to refine searches. Many of these methods are supported by the EUMETSAT Data Access Client (`eumdac`). In this notebook we will showcase some of the possibilities for using `eumdac` to better interact with SRAL collections. This notebook assumes that you already have an understanding of the available Data Store interfaces, which you can gain by running the **<a href="../1_SRAL_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>** notebook.


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Step 1: Authenticating the API](#section1)
1. [Step 2: Creating a search by collection](#section2)
1. [Step 3: Filtering by cycle number](#section3)
1. [Step 4: Filtering by timeliness](#section4)
1. [Step 5: Removing NTC duplicates](#section5)
1. [Step 6: Downloading the data](#section6)

<hr>

In [ ]:
import os
import json
import datetime
import shutil
import eumdac
import zipfile

# Create a download directory for our SLSTR products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Step 1: Authenticating the API
[Back to top](#TOC_TOP)

</div>

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token. More **essential** information on setting this up can be found in the **<a href="../1_OLCI_introductory/1_1_SRAL_data_access_HDA_Data_Store.ipynb">1_1_SRAL_data_access_HDA_Data_Store</a>** notebook.

In [ ]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>Step 2: Creating a search by collection
[Back to top](#TOC_TOP)

</div>

We will work with the SRAL Level-2 data throughout this notebook. The collection ID for this data type is `EO:EUM:DAT:0415`. You can find this information on the Data Store (https://data.eumetsat.int/), or ask the `eumdac` client to tell you all the avaiable collections by calling the `eumdac.DataStore(token).collections` method.

In [ ]:
# set collection ID for SRAL L2
collectionID = 'EO:EUM:DAT:0415'
cycle = 80

To filter by collection, we simply provide the collectionID to the `datastore.get_collection method`

In [ ]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(f"{selected_collection.title}\n---\n{selected_collection.abstract}")

<div class="alert alert-info" role="alert">

## <a id='section3'></a>Step 3: Filtering by cycle number
[Back to top](#TOC_TOP)

</div>

Now that we have our collection we can search within it using the cycle number to filter the results, as follows;

In [ ]:
products = selected_collection.search(
    cycle=cycle)
print(f"Found {len(products)} products")

<div class="alert alert-info" role="alert">

## <a id='section4'></a>Step 4: Filtering by timeliness
[Back to top](#TOC_TOP)

</div>

In this instance, we are only interested in Non Time-Critical (NTC) SRAL products. We can add this filter to our search as follows.

In [ ]:
products = selected_collection.search(
    cycle=cycle,
    timeliness="NT")
print(f"Found {len(products)} products")

<div class="alert alert-info" role="alert">

## <a id='section5'></a>Step 5: Removing NTC duplicates
[Back to top](#TOC_TOP)

</div>

Sometimes, during NTC processing, we have to revisit some granules to process them further. This is typically due to missing ancillary data that we have to wait for. In this case, we first pass and final pass products are both on the system. We only want the latter, final, product, which always has a later time. The box below will filter for the latest options only, taking advantage of the Data Stores default to show most recent products first.

In [ ]:
processed_list = []
final_products = []
for product in products:
    file_tags = str(product).split('_')
    file_tags = [i for i in file_tags if i]
    granule_start = file_tags[4]
    if granule_start not in processed_list:
        final_products.append(product)
        processed_list.append(granule_start)
        
print(f"Found {len(final_products)} products")

<div class="alert alert-info" role="alert">

## <a id='section6'></a>Step 6: Downloading the products
[Back to top](#TOC_TOP)

</div>

We have now narrowed our search to a the required products. We can now download them using the code below to write them into our specified directory and unzip them.

In [ ]:
max_products = 5
for final_product, count in zip(final_products, range(len(final_products))):
    if count >= max_products:
        continue
    print(f"Download {count}: {final_product}")
    with final_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(final_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {fdst.name} finished.')

    os.remove(fdst.name)

<div class="alert alert-block alert-warning">

### Challenge:

Now you have run this for acquiring the SRAL L2 data using the cycle number, can you adapt it for other orbital parameters?

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_1_SRAL_advanced_data_access_eumdac.ipynb" target="_blank"><< Advanced SRAL data access with the EUMDAC client</a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>